In [1]:
import numpy as np
import datetime, time, json
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, TimeDistributed, Dense, Lambda, concatenate, Dropout, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, History
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences
import pandas as pd

/Users/zhang/anaconda/envs/mlp/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
quora_corpus = np.load("/Users/zhang/MscProject_tweak2vec/corpus/quora_corpus_int5.npy")
labels = np.load("/Users/zhang/MscProject_tweak2vec/corpus/quora_labels.npy")

w2v_embedding = {}
w2v_embedding['pivots300_7m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_7m.npy")
w2v_embedding['pivots300_6m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_6m.npy")
w2v_embedding['pivots300_5m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_5m.npy")
w2v_embedding['pivots300_4m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_4m.npy")
w2v_embedding['pivots300_3m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_3m.npy")
w2v_embedding['pivots300_2m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_2m.npy")
w2v_embedding['pivots300_1m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_1m.npy")
w2v_embedding['pivots300_05m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_05m.npy")
w2v_embedding['pivots300_01m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_01m.npy")
w2v_embedding['pivots300_005m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_005m.npy")
w2v_embedding['pivots300_001m'] = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots300_001m.npy")

#w2v_concat = concat_vec = np.concatenate([w2v_google_50d, w2v_pivots100_50d], axis=1)
#w2v_avg = (w2v_google_50d + w2v_quora_50d)/2  

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
w2v_concat = pca.fit_transform(w2v_concat)
w2v_concat.shape

In [4]:
# separate question1 and question2
question1 = []
question2 = []
for n in range(int(len(quora_corpus)/2)):
    question1.append(quora_corpus[2*n])
    question2.append(quora_corpus[2*n+1])
    
q1_data = pad_sequences(question1, maxlen=25)
q2_data = pad_sequences(question2, maxlen=25)

In [5]:
l = 0
l_avg = []
for i in range(len(quora_corpus)):
    if len(quora_corpus[i]) > l:
        l = len(quora_corpus[i])
    l_avg.append(len(quora_corpus[i]))
print('max length:{0}, average length:{1}'.format(l,np.mean(np.array(l_avg))))

max length:190, average length:8.62675648734686


In [6]:
# hyperparameter setup
max_sentence_len = 25
embed_dim = 50
dropout_rate = 0.1
vocab_size = len(w2v_embedding['pivots300_7m'])

In [7]:
# split cross validation set and test set
questions = np.stack((q1_data, q2_data), axis=1)
X_train, X_test, y_train, y_test = train_test_split(questions, labels, test_size=0.1, random_state=2018)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]

### define the model

In [8]:
def Max_BoE(word_embedding):

    question1 = Input(shape=(max_sentence_len,))
    question2 = Input(shape=(max_sentence_len,))



    q1 = Embedding(  input_dim=vocab_size, 
                     output_dim=embed_dim, 
                     weights=[word_embedding], 
                     input_length=max_sentence_len, 
                     trainable=False)(question1)
    q1 = TimeDistributed(Dense(embed_dim, activation='relu'))(q1)
    q1 = Lambda(lambda x: K.max(x, axis=1), output_shape=(embed_dim, ))(q1)

    q2 = Embedding(  input_dim=vocab_size, 
                     output_dim=embed_dim, 
                     weights=[word_embedding], 
                     input_length=max_sentence_len, 
                     trainable=False)(question2)
    q2 = TimeDistributed(Dense(embed_dim, activation='relu'))(q2)
    q2 = Lambda(lambda x: K.max(x, axis=1), output_shape=(embed_dim, ))(q2)

    merged = concatenate([q1,q2])
    merged = Dense(200, activation='relu')(merged)
    merged = Dropout(dropout_rate)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(200, activation='relu')(merged)
    merged = Dropout(dropout_rate)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(200, activation='relu')(merged)
    merged = Dropout(dropout_rate)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(200, activation='relu')(merged)
    merged = Dropout(dropout_rate)(merged)
    merged = BatchNormalization()(merged)

    is_duplicate = Dense(1, activation='sigmoid')(merged)

    model = Model(inputs=[question1,question2], outputs=is_duplicate)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


### train the model

In [9]:
n_epoch = 50
val_split = 0.1
batch_size = 32

In [20]:
current_embed = np.load("/Users/zhang/MscProject_tweak2vec/word2vecModel/quora/tokens/w2v_pivots100_07m.npy")
model = Max_BoE(current_embed)
MODEL_WEIGHTS_FILE = '/Users/zhang/MscProject_tweak2vec/Max_BOE_weights/pivots100_07m.h5'
print("Starting training at", datetime.datetime.now())
t0 = time.time()
callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_acc', save_best_only=True)]
model_history = model.fit([Q1_train, Q2_train],
                    y_train,
                    epochs=n_epoch,
                    validation_split=val_split,
                    verbose=1,
                    batch_size=batch_size,
                    callbacks=callbacks)
data = model_history.history
t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))
print("-------------------------------------------------------------------------") 
print("-------------------------------------------------------------------------")

Starting training at 2018-07-15 14:00:01.172311
Train on 327472 samples, validate on 36386 samples
Epoch 1/50
327472/327472 [==============================] - 113s 344us/step - loss: 0.5560 - acc: 0.7162 - val_loss: 0.5088 - val_acc: 0.7482
Epoch 2/50
327472/327472 [==============================] - 111s 339us/step - loss: 0.5129 - acc: 0.7442 - val_loss: 0.4905 - val_acc: 0.7579
Epoch 3/50
327472/327472 [==============================] - 98s 301us/step - loss: 0.4976 - acc: 0.7531 - val_loss: 0.4854 - val_acc: 0.7605
Epoch 4/50
327472/327472 [==============================] - 112s 343us/step - loss: 0.4876 - acc: 0.7590 - val_loss: 0.4763 - val_acc: 0.7660
Epoch 5/50
327472/327472 [==============================] - 100s 306us/step - loss: 0.4812 - acc: 0.7625 - val_loss: 0.4717 - val_acc: 0.7679
Epoch 6/50
327472/327472 [==============================] - 97s 297us/step - loss: 0.4764 - acc: 0.7658 - val_loss: 0.4663 - val_acc: 0.7731
Epoch 7/50
327472/327472 [=========================

In [19]:
f = open('pivots100_07m_data.txt','w')
f.write(str(save_data))
f.close()

In [10]:
save_data = {}
for embed_name in w2v_embedding.keys():
    current_embed = w2v_embedding[embed_name]
    model = Max_BoE(current_embed)
    MODEL_WEIGHTS_FILE = '/Users/zhang/MscProject_tweak2vec/Max_BOE_weights/'+embed_name+'_weights.h5'
    print('current embedding: ',embed_name)
    print("Starting training at", datetime.datetime.now())
    t0 = time.time()
    callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_acc', save_best_only=True)]
    model_history = model.fit([Q1_train, Q2_train],
                        y_train,
                        epochs=n_epoch,
                        validation_split=val_split,
                        verbose=1,
                        batch_size=batch_size,
                        callbacks=callbacks)
    save_data[embed_name] = model_history.history
    t1 = time.time()
    print("Training ended at", datetime.datetime.now())
    print("Minutes elapsed: %f" % ((t1 - t0) / 60.))
    print("-------------------------------------------------------------------------") 
    print("-------------------------------------------------------------------------")

current embedding:  pivots300_7m
Starting training at 2018-07-13 19:03:38.603639
Train on 327472 samples, validate on 36386 samples
Epoch 1/50
327472/327472 [==============================] - 73s 222us/step - loss: 0.5508 - acc: 0.7188 - val_loss: 0.4981 - val_acc: 0.7541
Epoch 2/50
327472/327472 [==============================] - 67s 205us/step - loss: 0.5021 - acc: 0.7511 - val_loss: 0.4774 - val_acc: 0.7633
Epoch 3/50
327472/327472 [==============================] - 62s 188us/step - loss: 0.4833 - acc: 0.7620 - val_loss: 0.4627 - val_acc: 0.7726
Epoch 4/50
327472/327472 [==============================] - 62s 188us/step - loss: 0.4736 - acc: 0.7679 - val_loss: 0.4528 - val_acc: 0.7794
Epoch 5/50
327472/327472 [==============================] - 62s 188us/step - loss: 0.4664 - acc: 0.7727 - val_loss: 0.4520 - val_acc: 0.7802
Epoch 6/50
327472/327472 [==============================] - 61s 188us/step - loss: 0.4599 - acc: 0.7761 - val_loss: 0.4466 - val_acc: 0.7849
Epoch 7/50
327472/3274

327472/327472 [==============================] - 70s 213us/step - loss: 0.4670 - acc: 0.7713 - val_loss: 0.4521 - val_acc: 0.7782
Epoch 6/50
327472/327472 [==============================] - 71s 218us/step - loss: 0.4611 - acc: 0.7748 - val_loss: 0.4499 - val_acc: 0.7788
Epoch 7/50
327472/327472 [==============================] - 68s 209us/step - loss: 0.4560 - acc: 0.7781 - val_loss: 0.4395 - val_acc: 0.7881
Epoch 8/50
327472/327472 [==============================] - 67s 204us/step - loss: 0.4515 - acc: 0.7806 - val_loss: 0.4402 - val_acc: 0.7888
Epoch 9/50
327472/327472 [==============================] - 67s 206us/step - loss: 0.4470 - acc: 0.7842 - val_loss: 0.4397 - val_acc: 0.7884
Epoch 10/50
327472/327472 [==============================] - 66s 203us/step - loss: 0.4450 - acc: 0.7852 - val_loss: 0.4412 - val_acc: 0.7879
Epoch 11/50
327472/327472 [==============================] - 71s 216us/step - loss: 0.4411 - acc: 0.7865 - val_loss: 0.4399 - val_acc: 0.7893
Epoch 12/50
327472/327

327472/327472 [==============================] - 70s 215us/step - loss: 0.4505 - acc: 0.7814 - val_loss: 0.4445 - val_acc: 0.7866
Epoch 11/50
327472/327472 [==============================] - 73s 224us/step - loss: 0.4473 - acc: 0.7823 - val_loss: 0.4401 - val_acc: 0.7853
Epoch 12/50
327472/327472 [==============================] - 70s 214us/step - loss: 0.4448 - acc: 0.7827 - val_loss: 0.4473 - val_acc: 0.7815
Epoch 13/50
327472/327472 [==============================] - 72s 220us/step - loss: 0.4421 - acc: 0.7854 - val_loss: 0.4433 - val_acc: 0.7816
Epoch 14/50
327472/327472 [==============================] - 70s 214us/step - loss: 0.4399 - acc: 0.7883 - val_loss: 0.4372 - val_acc: 0.7891
Epoch 15/50
327472/327472 [==============================] - 65s 198us/step - loss: 0.4386 - acc: 0.7882 - val_loss: 0.4367 - val_acc: 0.7907
Epoch 16/50
327472/327472 [==============================] - 61s 187us/step - loss: 0.4360 - acc: 0.7899 - val_loss: 0.4408 - val_acc: 0.7870
Epoch 17/50
327472

327472/327472 [==============================] - 72s 219us/step - loss: 0.4362 - acc: 0.7907 - val_loss: 0.4326 - val_acc: 0.7906
Epoch 16/50
327472/327472 [==============================] - 68s 208us/step - loss: 0.4344 - acc: 0.7923 - val_loss: 0.4339 - val_acc: 0.7921
Epoch 17/50
327472/327472 [==============================] - 69s 212us/step - loss: 0.4331 - acc: 0.7931 - val_loss: 0.4355 - val_acc: 0.7920
Epoch 18/50
327472/327472 [==============================] - 70s 212us/step - loss: 0.4310 - acc: 0.7944 - val_loss: 0.4331 - val_acc: 0.7927
Epoch 19/50
327472/327472 [==============================] - 69s 212us/step - loss: 0.4302 - acc: 0.7948 - val_loss: 0.4314 - val_acc: 0.7946
Epoch 20/50
327472/327472 [==============================] - 70s 214us/step - loss: 0.4290 - acc: 0.7953 - val_loss: 0.4328 - val_acc: 0.7917
Epoch 21/50
327472/327472 [==============================] - 72s 219us/step - loss: 0.4276 - acc: 0.7960 - val_loss: 0.4319 - val_acc: 0.7920
Epoch 22/50
327472

327472/327472 [==============================] - 73s 224us/step - loss: 0.4315 - acc: 0.7935 - val_loss: 0.4334 - val_acc: 0.7938
Epoch 21/50
327472/327472 [==============================] - 74s 226us/step - loss: 0.4303 - acc: 0.7935 - val_loss: 0.4363 - val_acc: 0.7892
Epoch 22/50
327472/327472 [==============================] - 76s 232us/step - loss: 0.4284 - acc: 0.7956 - val_loss: 0.4327 - val_acc: 0.7953
Epoch 23/50
327472/327472 [==============================] - 78s 237us/step - loss: 0.4282 - acc: 0.7957 - val_loss: 0.4306 - val_acc: 0.7965
Epoch 24/50
327472/327472 [==============================] - 75s 229us/step - loss: 0.4275 - acc: 0.7962 - val_loss: 0.4345 - val_acc: 0.7912
Epoch 25/50
327472/327472 [==============================] - 75s 229us/step - loss: 0.4271 - acc: 0.7957 - val_loss: 0.4325 - val_acc: 0.7944
Epoch 26/50
327472/327472 [==============================] - 74s 226us/step - loss: 0.4243 - acc: 0.7982 - val_loss: 0.4310 - val_acc: 0.7942
Epoch 27/50
327472

...

327472/327472 [==============================] - 83s 253us/step - loss: 0.4316 - acc: 0.7925 - val_loss: 0.4400 - val_acc: 0.7890
Epoch 31/50
327472/327472 [==============================] - 85s 259us/step - loss: 0.4300 - acc: 0.7935 - val_loss: 0.4415 - val_acc: 0.7873
Epoch 32/50
327472/327472 [==============================] - 84s 255us/step - loss: 0.4296 - acc: 0.7934 - val_loss: 0.4417 - val_acc: 0.7865
Epoch 33/50
327472/327472 [==============================] - 82s 250us/step - loss: 0.4286 - acc: 0.7941 - val_loss: 0.4391 - val_acc: 0.7879
Epoch 34/50
327472/327472 [==============================] - 83s 253us/step - loss: 0.4293 - acc: 0.7935 - val_loss: 0.4424 - val_acc: 0.7833
Epoch 35/50
327472/327472 [==============================] - 84s 255us/step - loss: 0.4273 - acc: 0.7948 - val_loss: 0.4387 - val_acc: 0.7887
Epoch 36/50
327472/327472 [==============================] - 83s 253us/step - loss: 0.4273 - acc: 0.7947 - val_loss: 0.4374 - val_acc: 0.7895
Epoch 37/50
327472

327472/327472 [==============================] - 68s 208us/step - loss: 0.4285 - acc: 0.7949 - val_loss: 0.4393 - val_acc: 0.7887
Epoch 36/50
327472/327472 [==============================] - 68s 208us/step - loss: 0.4283 - acc: 0.7962 - val_loss: 0.4437 - val_acc: 0.7868
Epoch 37/50
327472/327472 [==============================] - 69s 209us/step - loss: 0.4277 - acc: 0.7952 - val_loss: 0.4466 - val_acc: 0.7835
Epoch 38/50
327472/327472 [==============================] - 68s 209us/step - loss: 0.4264 - acc: 0.7966 - val_loss: 0.4425 - val_acc: 0.7848
Epoch 39/50
327472/327472 [==============================] - 68s 209us/step - loss: 0.4254 - acc: 0.7966 - val_loss: 0.4397 - val_acc: 0.7883
Epoch 40/50
327472/327472 [==============================] - 68s 209us/step - loss: 0.4257 - acc: 0.7967 - val_loss: 0.4381 - val_acc: 0.7893
Epoch 41/50
327472/327472 [==============================] - 68s 209us/step - loss: 0.4258 - acc: 0.7966 - val_loss: 0.4403 - val_acc: 0.7882
Epoch 42/50
327472

327472/327472 [==============================] - 70s 214us/step - loss: 0.4373 - acc: 0.7899 - val_loss: 0.4545 - val_acc: 0.7790
Epoch 41/50
327472/327472 [==============================] - 70s 213us/step - loss: 0.4346 - acc: 0.7914 - val_loss: 0.4495 - val_acc: 0.7824
Epoch 42/50
327472/327472 [==============================] - 70s 213us/step - loss: 0.4338 - acc: 0.7922 - val_loss: 0.4464 - val_acc: 0.7846
Epoch 43/50
327472/327472 [==============================] - 70s 214us/step - loss: 0.4342 - acc: 0.7914 - val_loss: 0.4472 - val_acc: 0.7849
Epoch 44/50
327472/327472 [==============================] - 69s 211us/step - loss: 0.4324 - acc: 0.7923 - val_loss: 0.4503 - val_acc: 0.7823
Epoch 45/50
327472/327472 [==============================] - 70s 213us/step - loss: 0.4317 - acc: 0.7924 - val_loss: 0.4490 - val_acc: 0.7854
Epoch 46/50
327472/327472 [==============================] - 70s 214us/step - loss: 0.4316 - acc: 0.7932 - val_loss: 0.4455 - val_acc: 0.7840
Epoch 47/50
327472

327472/327472 [==============================] - 71s 216us/step - loss: 0.4415 - acc: 0.7870 - val_loss: 0.4537 - val_acc: 0.7792
Epoch 46/50
327472/327472 [==============================] - 71s 217us/step - loss: 0.4414 - acc: 0.7873 - val_loss: 0.4547 - val_acc: 0.7794
Epoch 47/50
327472/327472 [==============================] - 71s 217us/step - loss: 0.4407 - acc: 0.7877 - val_loss: 0.4532 - val_acc: 0.7808
Epoch 48/50
327472/327472 [==============================] - 70s 214us/step - loss: 0.4398 - acc: 0.7883 - val_loss: 0.4547 - val_acc: 0.7806
Epoch 49/50
327472/327472 [==============================] - 70s 214us/step - loss: 0.4382 - acc: 0.7891 - val_loss: 0.4530 - val_acc: 0.7821
Epoch 50/50
327472/327472 [==============================] - 70s 214us/step - loss: 0.4389 - acc: 0.7889 - val_loss: 0.4555 - val_acc: 0.7829
Training ended at 2018-07-14 05:10:02.774966
Minutes elapsed: 58.710928
-------------------------------------------------------------------------
--------------

327472/327472 [==============================] - 73s 222us/step - loss: 0.6590 - acc: 0.6303 - val_loss: 0.6560 - val_acc: 0.6355
Training ended at 2018-07-14 06:10:50.125781
Minutes elapsed: 60.717328
-------------------------------------------------------------------------
-------------------------------------------------------------------------


In [11]:
f = open('pivots300_data.txt','w')
f.write(str(save_data))
f.close()

### Evaluate the model with best validation accuracy on the test partition

In [22]:
model.load_weights(MODEL_WEIGHTS_FILE)
loss, accuracy = model.evaluate([Q1_test, Q2_test], y_test, verbose=0)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))

loss = 0.4726, accuracy = 0.8038


In [27]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 25)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 25)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 25, 50)       1515000     input_5[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 25, 50)       1515000     input_6[0][0]                    
__________________________________________________________________________________________________
time_distr